## Imports

In [1]:
import numpy as np

## Network Class

In [154]:
"""
Simple_NN
Class to create a simple neural network
    Activation Function: sigmoid
    Learning Algorithm: stochastic gradient descent with backpropagation
    Cost Function: Mean squared error
"""
class Simple_NN(object):
    """ 
    INITIALIZE THE NETWORK
    """
    def __init__(self, layers, activation_function="sigmoid"):
        """
        self.layers is a list of numbers where the ith number how many neurons are in
        the ith layer of the network.
        """
        self.layers = layers;
        self.num_layers = len(layers);
        
        """
        self.weights[Layer - 1, input_neuron, output_neuron] = 
                                            List of weight matrices for each layer.                      
        self.biases[Layer - 1, neuron] = 
                                            List of vectors with biases for each neuron  
        FOR EXAMPLE:
            self.weights[l, j, i] = weight going into the jth neuron of the lth layer
                                    from the ith neuron of the (l-1)st layer 
            self.biases[l, k] = bias on the kth nuron of the lth layer
        NOTE: layer 0 is the input layer, so self.weights[0] is the weights going into layer 1
        """
        self.weights = [];
        self.biases = [];
        
        self.Z = [];
        self.activations = [];
        # Create matrices with correct dimensions 
        for layer_num in range(1, self.num_layers):
            self.weights.append(np.random.randn(layers[layer_num], layers[layer_num - 1]));
            self.biases.append(np.random.randn(layers[layer_num]));
        """
        self.activation = string specifying what activation function the neurons will use. 
        The options are:
            sigmoid (default)
        """
        self.activation_function = activation_function;
        
    """ 
    ACTIVATION FUNCTION
    For this network, we use the sigmoid function to calculate neuron activation
    """      
    def activation(self, z):
        if (self.activation_function == "sigmoid"):
            return 1.0 / (1 + np.exp(-z));
    
    def activation_derivative(self, z):
        if (self.activation_function == "sigmoid"):
            return (1 - self.activation(z)) * self.activation(z);
        
    """
    TRAINING
    Train the network using stochastic gradient descent and backpropagation.
    Training data should be given in the following format:
        [x11, x12, ..., x1i, y1
         x21, x22, ..., x2i, y2
         ...
         xm1, xm1, ..., xmi, ym]
    Where each row corrsponds to a training example with i data points
    """
    def train(self, training_data, batch_size, num_epochs, learning_rate):
        for epoch in range(1, num_epochs):
            # Randomize the order of training examples
            np.random.shuffle(training_data);
            # Separate inputs from outputs
            inputs = training_data[:, :-2];
            outputs = training_data[:, -1];
            # For each epoch, loop through each batch to use as training data
            for batch in range(len(training_data))[0 :: batch_size]:
                # Create matrix out of all training inputs in the batch
                X = np.matrix(inputs[batch : batch + batch_size, :]); 
                Y = np.matrix(outputs[batch : batch + batch_size]);
                # If the first layer of the network has k neurons, and each training
                # example has i data points, then weights will be a kxi matrix 
                # so Wx_j = kx1 vector.
                # To apply W to all input vectors, we can multiply WX where
                # X is the ixm matrix containing all m training examples as columns
                
                print(X);
                #X = np.transpose(X);
                
                # FEEDFORWARD
                """
                self.Z[layer, training_example, neuron] = 
                                            List of vectors with weighted inputs to the neurons
                self.activations[layer, training_example, neuron] = 
                                            List of vectors with activations for each neuron
                """
                # Calclate outputs going forwards through the network
                for layer in range(self.num_layers - 1):
                    if layer == 0:
                        # Feed inputs to the network
                        prev_activations = X;
                    else:
                        prev_activations = self.activations[layer - 1];
                    # Bias matrix where each column is a copy of the bias vector is needed
                    # to add bias terms for each training example. 
                    one_vector = np.ones(len(self.biases[layer]));
                    bias_matrix = np.outer(self.biases[layer], one_vector);
                    self.Z.append(np.dot(self.weights[layer], prev_activations) + \
                                  bias_matrix);
                    self.activations.append(self.activation(self.Z[layer]));
            
                # Backpropagation
                pass
                # Gradient Descent
    
    """ 
    TODO: 
        test(testing_data)
    """
    def print_network(self):
        print("Weights: ")
        for layer in self.weights:
            print(layer)
        print("\nBiases:" )
        for layer in self.biases:
            print(layer)
        print("\nWeighted Inputs:")
        for layer in self.Z:
            print(layer)
        print("\nActivations:")
        for layer in self.activations:
            print(layer);

In [156]:
"""
TEST NETWORK CREATION
"""
test = Simple_NN([3, 5, 2]);
test.print_network();
print("\n")

random_data = np.matrix('1, 2, 3, 4, 20; 11, 12, 13, 14, 30; 21, 22, 23, 24, 40')
test.train(random_data, 1, 3, 0.5);



Weights: 
[[-2.65780809 -1.58207156 -0.45558872]
 [ 1.87957975  0.1496402  -0.22577234]
 [-0.28291263 -1.49836166 -1.19630648]
 [ 0.58258132 -0.21383262  1.20064711]
 [-0.31830545  0.08265716 -0.80310833]]
[[ 1.69679628  1.78833209 -0.85038455  2.74153751  1.44638488]
 [-0.5515198   1.04475691  1.15470594  1.14839571  0.4113301 ]]

Biases:
[-1.99461823  0.29609005 -1.91517441  0.40174685  0.0855234 ]
[-0.300632   -1.14226778]

Weighted Inputs:

Activations:


[[1 2 3]]


ValueError: shapes (5,3) and (1,3) not aligned: 3 (dim 1) != 1 (dim 0)